# PlayHQ Fixture Scraping

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ssardina/tapp-fixture/blob/main/playhq_scrape.ipynb)

This system allows to scrape game fixtures from [PlayHQ](http://playhq.com/) via its Public [API](https://support.playhq.com/hc/en-au/sections/4405422358297-PlayHQ-APIs). It will produce a CSV file ready to be uploaded as Schedule in [TeamApp](https://brunswickmagicbasketball.teamapp.com/).

The *Public* APIs only require a header parameters to get a successful response, which includes the following components:

- `x-api-key` (also referred to as the Client ID) will be provided by PlayHQ when you request access to the public API via their [support page](https://support.playhq.com/hc/en-au) or email support@playhqsupport.zendesk.com. This key can be stored in a file `x_api_key.txt` or it will be asked interactively by the notebook otherwise. In many cases, the feature to create new API credentials is disabled for a user and can only be actioned by a Super Administrator role within the Play HQ portal.
- `x-phq-tenant` usually refers to the sport/association - in this case '`bv`'.


Detailed reference documentation for PlayHQ API can be found [here](https://docs.playhq.com/tech).

**Contact:** Sebastian Sardina (sssardina@gmail.com)

In [1]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import re
import os
import calendar, datetime
import dtale

import utils
import playhq as phq

## 1. Configuration and set-up

We first configure and set-up the application. This means reading configuration variables from a config file and setting the game day.

So, first of all, specify the following information:

1. Configuration file for the club and season.
2. Game dates interval to scrape.

In [2]:
# Change this to import your club's own configuration
from config_bmc_w23 import *
# from config_cba import *

# Set the game date interval scrape
# GAME_DATE_START = datetime.date.today() # by default, any game after today
# GAME_DATE_START = utils.next_day(calendar.SATURDAY)   # start from next game day
GAME_DATE_START = datetime.date(2023, 9, 6) # start on a specific day - REP ROUND 5+

WEEKS = 1   # how many weeks after date start we want to scrape (use 1 for just next game)
GAME_DATE_END = GAME_DATE_START + datetime.timedelta(days=WEEKS*7)


###############################################################
# DO NOT CHANGE FROM HERE
###############################################################

# Get nice game date format: Saturday August 06, 2022
GAME_DATE_START_TIMESTAMP = pd.to_datetime(GAME_DATE_START).tz_localize(TIMEZONE)
GAME_DATE_END_TIMESTAMP = pd.to_datetime(GAME_DATE_END).tz_localize(TIMEZONE)

GAME_DATE_START_NAME = utils.pretty_date(GAME_DATE_START_TIMESTAMP)
GAME_DATE_END_NAME = utils.pretty_date(GAME_DATE_END_TIMESTAMP)

# Create phq_club object
phq_club = phq.PlayHQ(CLUB_NAME, ORG_ID, X_API_KEY, X_TENANT, TIMEZONE, tapp_team_name, tapp_game_name)
if SEASON_ID is None:
    raise SystemExit("ERROR! Please specify either SEASON_ID.")
SEASON_NAME = phq_club.get_season_name(SEASON_ID)

print(f"Club name: {CLUB_NAME} (org. id: {ORG_ID})")
print(f"Season: {SEASON_NAME} (season id: {SEASON_ID})")
print("X-tenant:", X_TENANT, "x-api-key:", X_API_KEY)
print("Output path:", OUTPUT_PATH)
if not os.path.exists(OUTPUT_PATH):
    raise SystemExit("ERROR! Output path {OUTPUT_PATH} is missing! Please create or link that path correctly to save data.")

print("Timezone:", TIMEZONE)
print(f"Game dates: {GAME_DATE_START_NAME} - {GAME_DATE_END_NAME}")
print("PlayHQ Club fixture:", PLAYHQ_SEASON_URL)

Club name: Brunswick Magic Basketball Club (org. id: 8c4d5431-eaa5-4644-82ac-992abe224b88)
Season: Junior Domestic (season id: cdbe3065-2a32-4c6d-8771-f8fae3fa7611)
X-tenant: bv x-api-key: f5d33c76-f858-49fa-8330-8e0e396219cd
Output path: Brunswick_Magics/2023.01.Winter-23/fixture/
Timezone: Australia/Melbourne
Game dates: Wednesday September 06, 2023 (2023/09/06) - Wednesday September 13, 2023 (2023/09/13)
PlayHQ Club fixture: https://bit.ly/bmbc-w23


## 2. Get upcoming games for club's teams

First, get the teams of the club, sort them based on age group.

In [3]:
teams_df = phq_club.get_season_teams(SEASON_ID)
teams_df.sort_values('age', ascending=True, inplace=True)
teams_df.reset_index(inplace=True, drop=True)

teams = teams_df['name'].values
print(f"Found {len(teams)} teams:", teams)

# teams_df
# teams_df.query("name == 'Coburg U12 Boys 1'")

# if needed keep only a specific team (e.g., for debugging later)
# teams_df = teams_df.query("id == '115ca2a9-5383-4beb-9703-691006568974'")

Found 28 teams: ['Magic U10 Boys Black' 'Magic U10 Boys Gold' 'Magic U10 Boys Purple'
 'Magic U10 Girls Black' 'Magic U10 Girls Gold' 'Magic U10 Girls Purple'
 'Magic U12 Boys Black' 'Magic U12 Boys Diamond' 'Magic U12 Boys Gold'
 'Magic U12 Boys Purple' 'Magic U12 Girls Black' 'Magic U12 Girls Gold'
 'Magic U12 Girls Purple' 'Magic U14 Girls Purple' 'Magic U14 Girls Black'
 'Magic U14 Boys Purple' 'Magic U14 Boys Black' 'Magic U14 Boys Diamond'
 'Magic U14 Boys Gold' 'Magic U16 Boys Black' 'Magic U16 Boys Diamond'
 'Magic U16 Boys Gold' 'Magic U16 Boys Purple' 'Magic U16 Girls Purple'
 'Magic U18 Boys Purple' 'Magic U18 Girls Gold' 'Magic U8 Mixed Gold'
 'Magic U8 Mixed Purple']


Next, extract all games between the dates specified for these teams of the club.

In [4]:
upcoming_games_df = phq_club.get_games(teams_df, GAME_DATE_START_TIMESTAMP, GAME_DATE_END_TIMESTAMP)

if upcoming_games_df is not None:
    print(f'There were {upcoming_games_df.shape[0]} games extracted for game between {GAME_DATE_START_NAME} and {GAME_DATE_END_NAME}')
    upcoming_games_df[phq.GAMES_COLS]
else:
    print(f'No games between {GAME_DATE_START_NAME} and {GAME_DATE_END_NAME}')

2023-09-06 14:25:34 INFO Games extracted for team: Magic U10 Boys Black
2023-09-06 14:25:34 INFO Games extracted for team: Magic U10 Boys Gold
2023-09-06 14:25:34 INFO Games extracted for team: Magic U10 Boys Purple
2023-09-06 14:25:35 INFO Games extracted for team: Magic U10 Girls Black
2023-09-06 14:25:35 INFO Games extracted for team: Magic U10 Girls Gold
2023-09-06 14:25:35 INFO Games extracted for team: Magic U10 Girls Purple
2023-09-06 14:25:36 INFO Games extracted for team: Magic U12 Boys Black
2023-09-06 14:25:36 INFO Games extracted for team: Magic U12 Boys Diamond
2023-09-06 14:25:36 INFO Games extracted for team: Magic U12 Boys Gold
2023-09-06 14:25:37 WARNING Games with ids ['4210d292-80a3-456a-9a9d-5272e4a78f2c'] have no date. They will be dropped
2023-09-06 14:25:37 INFO No games for team: Magic U12 Boys Purple
2023-09-06 14:25:37 INFO No games for team: Magic U12 Girls Black
2023-09-06 14:25:37 INFO Games extracted for team: Magic U12 Girls Gold
2023-09-06 14:25:38 INFO 

There were 16 games extracted for game between Wednesday September 06, 2023 (2023/09/06) and Wednesday September 13, 2023 (2023/09/13)


In [5]:
# dtale.show(upcoming_games_df)
upcoming_games_df

,id,team_name,team_id,status,url,createdAt,updatedAt,pool,competitors,grade_id,...,venue_surfaceAbbreviation,venue_address_line1,venue_address_postcode,venue_address_suburb,venue_address_state,venue_address_country,venue_address_latitude,venue_address_longitude,schedule_timestamp,venue
0,e716cf7d-bd47-4fe5-9b93-9c713f18d1bc,Magic U10 Boys Black,6a8d8b03-f1af-4a33-bca5-823175f07be4,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2023-09-04 14:33:28+10:00,2023-09-04 19:45:07+10:00,None,[{'id': '6a8d8b03-f1af-4a33-bca5-823175f07be4'...,db7d3614-4ac2-4d1d-b3cd-e8250cfb8b6a,...,Crt2,25 Outlook Road,3058,Coburg North,VIC,Australia,-37.733150,144.976840,2023-09-09 09:30:00+10:00,NaN
1,f2039830-96a8-4da0-b243-92f0651716f1,Magic U10 Boys Gold,270117c1-19e8-4d81-8fad-084a6dd8b5c3,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2023-09-04 14:33:28+10:00,2023-09-04 19:45:06+10:00,None,[{'id': '270117c1-19e8-4d81-8fad-084a6dd8b5c3'...,a1fde50a-d092-4a18-9be1-103a5a5da01f,...,Crt1,"BOX FOREST COLLEGE, 120 GLENROY RD",3046,GLENROY,VIC,Australia,-37.704593,144.925895,2023-09-09 09:30:00+10:00,NaN
2,3cd51835-e4b1-4f87-9728-cd9b313ba37f,Magic U10 Boys Purple,5e402da3-19fb-4c51-956c-e9988f3154c0,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2023-09-04 14:33:28+10:00,2023-09-04 18:42:35+10:00,None,[{'id': '5e402da3-19fb-4c51-956c-e9988f3154c0'...,669841ed-b349-4821-8a60-28faa8eabb4f,...,Crt1,21 Railway Place West,3072,Preston,VIC,Australia,-37.748450,144.999440,2023-09-09 09:30:00+10:00,NaN
3,ca082367-2879-426b-a7e9-3e0f5f625b5f,Magic U10 Girls Black,d81ed8d3-e1cb-4a1c-bd50-e198c41c9234,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2023-09-04 14:33:28+10:00,2023-09-04 18:22:49+10:00,None,[{'id': 'd81ed8d3-e1cb-4a1c-bd50-e198c41c9234'...,dfe76b59-30fe-44e3-8f0d-388a5dbd857d,...,Crt1,"BOX FOREST COLLEGE, 120 GLENROY RD",3046,GLENROY,VIC,Australia,-37.704593,144.925895,2023-09-09 08:30:00+10:00,NaN
4,68f3b191-4c00-47c7-be19-de82f4557e3f,Magic U10 Girls Gold,eae85f9a-76ae-47bb-9a4b-129332952379,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2023-09-04 14:33:28+10:00,2023-09-04 18:42:39+10:00,None,[{'id': 'e619fe6c-1961-427d-abf6-24170a2061e7'...,dfe76b59-30fe-44e3-8f0d-388a5dbd857d,...,Crt1,9 Hillcrest Road,3046,Oak Park,VIC,Australia,-37.713213,144.925617,2023-09-09 09:30:00+10:00,NaN
5,6f29274f-3971-4acf-a609-6d21015e5338,Magic U10 Girls Purple,67b48289-bc52-4379-92e2-f3a4a38ab71a,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2023-09-04 14:33:28+10:00,2023-09-05 09:27:12+10:00,None,[{'id': '67b48289-bc52-4379-92e2-f3a4a38ab71a'...,dfe76b59-30fe-44e3-8f0d-388a5dbd857d,...,Crt1,101 Urquhart Street,3058,Coburg,VIC,Australia,-37.740720,144.971810,2023-09-09 09:30:00+10:00,NaN
6,74eddf27-d5c5-44c2-b24d-4d7feef53730,Magic U12 Boys Black,df9a835d-8622-4b3e-bf39-fb46acb045a3,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2023-08-29 20:15:58+10:00,2023-09-04 19:25:13+10:00,None,[{'id': 'fc40ead3-4e67-426a-85dc-0fa42b2231cc'...,19a88cf4-a5aa-4dd8-996a-28b46eec71aa,...,Crt1,760 Sydney Road,3058,Coburg,VIC,Australia,-37.729810,144.964030,2023-09-09 10:30:00+10:00,NaN
7,94025e1e-07df-4722-b301-56023e1698f2,Magic U12 Boys Diamond,7be8ced6-20fa-4698-b9d3-30b43bbc716d,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2023-08-29 20:20:51+10:00,2023-09-04 19:45:10+10:00,None,[{'id': '7be8ced6-20fa-4698-b9d3-30b43bbc716d'...,ea446b3f-d1dd-48e2-abf3-7bd7eee6af19,...,Crt1,21 Railway Place West,3072,Preston,VIC,Australia,-37.748450,144.999440,2023-09-09 10:30:00+10:00,NaN
8,90c9a861-d8bf-4cb1-ae90-c76e28415478,Magic U12 Boys Gold,d7754e94-0307-4d47-bc51-bf0e888412c4,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2023-08-29 20:20:00+10:00,2023-09-04 19:45:10+10:00,None,[{'id': 'ca5e96ea-bd2f-4130-a685-85ab643669d7'...,39c08b7d-58dc-4f1d-890e-0188cdcc56fd,...,Crt1,"BOX FOREST COLLEGE, 120 GLENROY RD",3046,GLENROY,VIC,Australia,-37.704593,144.925895,2023-09-09 11:30:00

## 3. Convert to TeamApp CSV format

Next, we convert the PlayHQ upcoming games to Teams App format so we can produce a CSV file to be imported into Teams App.

This process takes time as it processes game per game and even obtains short URL links for each game.

In [6]:
games_tapps_df = phq_club.to_teamsapp_schedule(upcoming_games_df, desc_template=DESC_TAPP, game_duration=45)
print("Done computing the games for Teams App")

# find out the game day if there is one day all teams play on only
game_day = None
single_game_day = (games_tapps_df['start_date'].drop_duplicates().size == 1)
if single_game_day:
    game_day = games_tapps_df.iloc[0]['start_date']
    print("All games are in the following day:", utils.pretty_date(game_day))

Done computing the games for Teams App
All games are in the following day: Saturday September 09, 2023 (2023/09/09)


Inspect how the description of one of the games will look like:

In [7]:
# games_tapps_df.sample(3)
# dtale.show(games_tapps_df)
games_tapps_df

,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing,opponent,court
0,Game U10 Boys Black - Round 19,U10 Boys Black,2023-09-09,2023-09-09,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: STAR...,Coburg Basketball Stadium,"25 Outlook Road, Coburg North",U10 Boys Black,1,1,0,1,0,STARS U10 Boys GL,Court 2
1,Game U10 Boys Gold - Round 19,U10 Boys Gold,2023-09-09,2023-09-09,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: Newl...,Glenroy College,"BOX FOREST COLLEGE, 120 GLENROY RD, GLENROY",U10 Boys Gold,1,1,0,1,0,Newlands U10 Boys NUGGETS,Court 1
2,Game U10 Boys Purple - Round 19,U10 Boys Purple,2023-09-09,2023-09-09,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: Warr...,St John's College (Preston),"21 Railway Place West, Preston",U10 Boys Purple,1,1,0,1,0,Warriors U10 Boys Black,Court 1
3,Game U10 Girls Black - Round 19,U10 Girls Black,2023-09-09,2023-09-09,08:30:00,09:15:00,RSVP mandatory for the game.\n\nOpponent: Pant...,Glenroy College,"BOX FOREST COLLEGE, 120 GLENROY RD, GLENROY",U10 Girls Black,1,1,0,1,0,Panthers U10 Girls Black,Court 1
4,Game U10 Girls Gold - Round 19,U10 Girls Gold,2023-09-09,2023-09-09,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: St F...,Oak Park Stadium,"9 Hillcrest Road, Oak Park",U10 Girls Gold,1,1,0,1,0,St Fidelis U10 Girls Blue,Court 1
5,Game U10 Girls Purple - Round 19,U10 Girls Purple,2023-09-09,2023-09-09,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: Melb...,Coburg Senior High School,"101 Urquhart Street, Coburg",U10 Girls Purple,1,1,0,1,0,Melb Warriors U10 Girls,Court 1
6,Game U12 Boys Black - Finals Round 2,U12 Boys Black,2023-09-09,2023-09-09,10:30:00,11:15:00,RSVP mandatory for the game.\n\nOpponent: Warr...,Mercy College,"760 Sydney Road, Coburg",U12 Boys Black,1,1,0,1,0,Warriors U12 Boys Gold,Court 1
7,Game U12 Boys Diamond - Preliminary Finals,U12 Boys Diamond,2023-09-09,2023-09-09,10:30:00,11:15:00,RSVP mandatory for the game.\n\nOpponent: Newl...,St John's College (Preston),"21 Railway Place West, Preston",U12 Boys Diamond,1,1,0,1,0,Newlands U12 Boys KINGS,Court 1
8,Game U12 Boys Gold - Preliminary Finals,U12 Boys Gold,2023-09-09,2023-09-09,11:30:00,12:15:00,RSVP mandatory for the game.\n\nOpponent: Pira...,Glenroy College,"BOX FOREST COLLEGE, 120 GLENROY RD, GLENROY",U12 Boys Gold,1,1,0,1,0,Piranhas U12 Boys Neon,Court 1
9,Game U12 Girls Gold - Preliminary Finals,U12 Girls Gold,2023-09-09,2023-09-09,00:00:00,00:45:00,RSVP mandatory for the game.\n\nOpponent: Newl...,NaN,NaN,U12 Girls Gold,1,1,0,1,0,Newlands U12 Girls LEOPARDS,NaN


Keep games after a particular date:

In [8]:
import datetime

START_DATE = datetime.date(2023, 7, 21)
games_tapps_df[games_tapps_df['start_date'] > START_DATE]
games_tapps_df = games_tapps_df[games_tapps_df['start_date'] > START_DATE]
games_tapps_df

,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing,opponent,court
0,Game U10 Boys Black - Round 19,U10 Boys Black,2023-09-09,2023-09-09,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: STAR...,Coburg Basketball Stadium,"25 Outlook Road, Coburg North",U10 Boys Black,1,1,0,1,0,STARS U10 Boys GL,Court 2
1,Game U10 Boys Gold - Round 19,U10 Boys Gold,2023-09-09,2023-09-09,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: Newl...,Glenroy College,"BOX FOREST COLLEGE, 120 GLENROY RD, GLENROY",U10 Boys Gold,1,1,0,1,0,Newlands U10 Boys NUGGETS,Court 1
2,Game U10 Boys Purple - Round 19,U10 Boys Purple,2023-09-09,2023-09-09,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: Warr...,St John's College (Preston),"21 Railway Place West, Preston",U10 Boys Purple,1,1,0,1,0,Warriors U10 Boys Black,Court 1
3,Game U10 Girls Black - Round 19,U10 Girls Black,2023-09-09,2023-09-09,08:30:00,09:15:00,RSVP mandatory for the game.\n\nOpponent: Pant...,Glenroy College,"BOX FOREST COLLEGE, 120 GLENROY RD, GLENROY",U10 Girls Black,1,1,0,1,0,Panthers U10 Girls Black,Court 1
4,Game U10 Girls Gold - Round 19,U10 Girls Gold,2023-09-09,2023-09-09,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: St F...,Oak Park Stadium,"9 Hillcrest Road, Oak Park",U10 Girls Gold,1,1,0,1,0,St Fidelis U10 Girls Blue,Court 1
5,Game U10 Girls Purple - Round 19,U10 Girls Purple,2023-09-09,2023-09-09,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: Melb...,Coburg Senior High School,"101 Urquhart Street, Coburg",U10 Girls Purple,1,1,0,1,0,Melb Warriors U10 Girls,Court 1
6,Game U12 Boys Black - Finals Round 2,U12 Boys Black,2023-09-09,2023-09-09,10:30:00,11:15:00,RSVP mandatory for the game.\n\nOpponent: Warr...,Mercy College,"760 Sydney Road, Coburg",U12 Boys Black,1,1,0,1,0,Warriors U12 Boys Gold,Court 1
7,Game U12 Boys Diamond - Preliminary Finals,U12 Boys Diamond,2023-09-09,2023-09-09,10:30:00,11:15:00,RSVP mandatory for the game.\n\nOpponent: Newl...,St John's College (Preston),"21 Railway Place West, Preston",U12 Boys Diamond,1,1,0,1,0,Newlands U12 Boys KINGS,Court 1
8,Game U12 Boys Gold - Preliminary Finals,U12 Boys Gold,2023-09-09,2023-09-09,11:30:00,12:15:00,RSVP mandatory for the game.\n\nOpponent: Pira...,Glenroy College,"BOX FOREST COLLEGE, 120 GLENROY RD, GLENROY",U12 Boys Gold,1,1,0,1,0,Piranhas U12 Boys Neon,Court 1
9,Game U12 Girls Gold - Preliminary Finals,U12 Girls Gold,2023-09-09,2023-09-09,00:00:00,00:45:00,RSVP mandatory for the game.\n\nOpponent: Newl...,NaN,NaN,U12 Girls Gold,1,1,0,1,0,Newlands U12 Girls LEOPARDS,NaN


In [9]:
# Inspect description game of one team
# team = "20.2 Boys"
team = "U12 Boys Purple"

print("Description for:", team)
print(games_tapps_df.query("team_name == @team")['description'].values[0])

Description for: U12 Boys Purple


IndexError: index 0 is out of bounds for axis 0 with size 0

## 4. Append BYE games (if necessary)

We generate BYE entries for TeamsAPP ***only*** if the games are played all in the same day.

In [10]:
bye_teams= False    # assume no bye games

if game_day is not None:
    game_day = games_tapps_df.iloc[0]['start_date']

    # Extract the date of the round
    # date = games_tapps_df.iloc[1]['start_date']
    print(f"Extract BYE games for games on {utils.pretty_date(game_day)}")

    playing_teams = upcoming_games_df['team_id'].tolist()
    bye_teams = teams_df.loc[~teams_df['id'].isin(playing_teams)]['name'].tolist()
    bye_teams = list(map(lambda x: re.search("U.*", x).group(0), bye_teams))

    if bye_teams:
        games_bye_df = phq_club.build_teamsapp_bye_schedule(bye_teams, game_day, DESC_BYE_TAPP)
        print(f"Bye teams ({len(bye_teams)}): ", bye_teams)

        games_tapps_df = pd.concat([games_tapps_df, games_bye_df])
        games_tapps_df.drop_duplicates(inplace=True)
        games_tapps_df.reset_index(inplace=True, drop=True)
    else:
        print("No BYE games this round...")
else:
    print("Games obtained are not on the same day, not computing BYE games...")

(bye_teams and games_bye_df)

Extract BYE games for games on Saturday September 09, 2023 (2023/09/09)
Bye teams (12):  ['U12 Boys Purple', 'U12 Girls Black', 'U12 Girls Purple', 'U14 Girls Purple', 'U14 Girls Black', 'U14 Boys Purple', 'U14 Boys Black', 'U14 Boys Diamond', 'U14 Boys Gold', 'U16 Boys Diamond', 'U8 Mixed Gold', 'U8 Mixed Purple']


,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing
0,U12 Boys Purple - BYE,U12 Boys Purple,2023-09-09,2023-09-09,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Boys Purple,0,0,0,0,0
1,U12 Girls Black - BYE,U12 Girls Black,2023-09-09,2023-09-09,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Girls Black,0,0,0,0,0
2,U12 Girls Purple - BYE,U12 Girls Purple,2023-09-09,2023-09-09,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Girls Purple,0,0,0,0,0
3,U14 Girls Purple - BYE,U14 Girls Purple,2023-09-09,2023-09-09,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Girls Purple,0,0,0,0,0
4,U14 Girls Black - BYE,U14 Girls Black,2023-09-09,2023-09-09,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Girls Black,0,0,0,0,0
5,U14 Boys Purple - BYE,U14 Boys Purple,2023-09-09,2023-09-09,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Boys Purple,0,0,0,0,0
6,U14 Boys Black - BYE,U14 Boys Black,2023-09-09,2023-09-09,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Boys Black,0,0,0,0,0
7,U14 Boys Diamond - BYE,U14 Boys Diamond,2023-09-09,2023-09-09,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Boys Diamond,0,0,0,0,0
8,U14 Boys Gold - BYE,U14 Boys Gold,2023-09-09,2023-09-09,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Boys Gold,0,0,0,0,0
9,U16 Boys Diamond - BYE,U16 Boys Diamond,2023-09-09,2023-09-09,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U16 Boys Diamond,0,0,0,0,0


## 5. Final review

Finally, we report the games to be written into Schedule CSV file and **CHECKING THAT ALL IS GOOD TO GO!**

Particularly, look for games that are schedule but **PENDING** and without all details (time or venue).

In [11]:
games_tapps_df.columns
games_tapps_df[['event_name', 'team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']]
# games_tapps_df

,event_name,team_name,opponent,start_date,start_time,venue,court
0,Game U10 Boys Black - Round 19,U10 Boys Black,STARS U10 Boys GL,2023-09-09,09:30:00,Coburg Basketball Stadium,Court 2
1,Game U10 Boys Gold - Round 19,U10 Boys Gold,Newlands U10 Boys NUGGETS,2023-09-09,09:30:00,Glenroy College,Court 1
2,Game U10 Boys Purple - Round 19,U10 Boys Purple,Warriors U10 Boys Black,2023-09-09,09:30:00,St John's College (Preston),Court 1
3,Game U10 Girls Black - Round 19,U10 Girls Black,Panthers U10 Girls Black,2023-09-09,08:30:00,Glenroy College,Court 1
4,Game U10 Girls Gold - Round 19,U10 Girls Gold,St Fidelis U10 Girls Blue,2023-09-09,09:30:00,Oak Park Stadium,Court 1
5,Game U10 Girls Purple - Round 19,U10 Girls Purple,Melb Warriors U10 Girls,2023-09-09,09:30:00,Coburg Senior High School,Court 1
6,Game U12 Boys Black - Finals Round 2,U12 Boys Black,Warriors U12 Boys Gold,2023-09-09,10:30:00,Mercy College,Court 1
7,Game U12 Boys Diamond - Preliminary Finals,U12 Boys Diamond,Newlands U12 Boys KINGS,2023-09-09,10:30:00,St John's College (Preston),Court 1
8,Game U12 Boys Gold - Preliminary Finals,U12 Boys Gold,Piranhas U12 Boys Neon,2023-09-09,11:30:00,Glenroy College,Court 1
9,Game U12 Girls Gold - Preliminary Finals,U12 Girls Gold,Newlands U12 Girls LEOPARDS,2023-09-09,00:00:00,NaN,NaN


We stop the execution here if we are running all Jupyter notebook.

In [ ]:
raise SystemExit("Stop right there! Continue below to produce the CSV file if needed.")

## 6. Save to CSV file for Teams App import

OK we are ready to import into Teams App.

### 6.2. Check changes with previous saves

If the schedule was generated before, check if the new one differs with the one saved already.

First, let us define the files that we will save to disk.

In [ ]:
now = datetime.datetime.now() # current date and time
now_str = now.strftime("%Y_%m_%d-%H:%M:%S")

id_file = now_str
if game_day is not None:    # there is one date for all games!
    id_file = utils.compact_date(game_day)

file_csv = os.path.join(OUTPUT_PATH, f"schedule-teamsapp-{id_file}.csv")
file_upcoming_pkl = os.path.join(OUTPUT_PATH, f"upcoming_games_df-{id_file}.pkl")
file_games_tapps = os.path.join(OUTPUT_PATH, f"games_tapps_df-{id_file}.pkl")

print("Files to save:")
print(file_csv)
print(file_upcoming_pkl)
print(file_games_tapps)

if not os.path.exists(OUTPUT_PATH):
    raise SystemExit("ERROR! Output path {OUTPUT_PATH} is missing! Please create or link that path correctly to save data.")

Next, let's check if there was a saved file for the upcoming game day.

In [ ]:
cols = ['team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']

changed_games_df = None
if os.path.exists(file_games_tapps):
    print("There was already a schedule saved, recovering it to compare...")
    old_games_tapps_df = pd.read_pickle(file_games_tapps)

    teams_changed = pd.concat([games_tapps_df[cols], old_games_tapps_df[cols]]).drop_duplicates(keep=False)['team_name'].unique()
    print("Teams whose games have changed (updated, new, dropped):", teams_changed)

    old_games_df = old_games_tapps_df[cols].query("team_name in @teams_changed")
    new_games_df = games_tapps_df[cols].query("team_name in @teams_changed")
    changed_games_df = new_games_df.merge(old_games_df, how="inner", on="team_name", suffixes=('_new', '_old'))
else:
    print("No previous schedule saved")

# Show changes if any...
changed_games_df

### 5.3. Write a TeamAPP Schedule CSV & Datafarmes Pickles

Finally, we save the data to a CSV file that can be imported into the [SCHEDULE of TeamsApp for all Entries](https://brunswickmagicbasketball.teamapp.com/clubs/263995/events?_list=v1&team_id=all).

In [ ]:
import shutil

print('Saving TeamAPP schedule CSV file and Dataframes with id:', id_file)
for f in [file_csv, file_upcoming_pkl, file_games_tapps]:
  if os.path.exists(f):
    print("Backup file", f)
    shutil.copy(f, f + ".bak")

print('Saving CSV TeamApp schedule:', file_csv)
games_tapps_df.to_csv(file_csv, index=False)

print('Saving dataframe pickle:', file_upcoming_pkl)
upcoming_games_df.to_pickle(file_upcoming_pkl)
print('Saving dataframe pickle:', file_games_tapps)
games_tapps_df.to_pickle(file_games_tapps)

print(f"Finished saving CSV and DATA-FRAMNE files: {now.strftime('%d/%m/%Y, %H:%M:%S')}")

# ------------ END FIXTURE PUBLISHING ------------

### Check a particular team

In [ ]:
team = "U10 Girls Gold"

print(games_tapps_df.query("team_name == @team")['description'].values[0])
games_tapps_df.query("team_name == @team")[['team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']]
